The code is ChatGPT generated as far as pygame is concerned but the thought process is mine :)(I don't know pygame much)

In [1]:
import pygame
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Initialize Pygame and set up fullscreen
pygame.init()
infoObject = pygame.display.Info()
SCREEN_WIDTH = infoObject.current_w
SCREEN_HEIGHT = infoObject.current_h
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT), pygame.FULLSCREEN)

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)

# Fonts
FONT = pygame.font.Font(None, 36)
SCORE_FONT = pygame.font.Font(None, 48)

# Game variables
round_num = 1
player_actions = []
opponent_actions = []
results = []

# Scores
player_score = 0
opponent_score = 0

# Cheat limits
player_cheats = 0
opponent_cheats = 0
CHEAT_LIMIT = 25

# Initialize Linear Regression model
model = LinearRegression()

# Predefine rounds
all_rounds = list(range(1, 101))

# Simulate initial data for AI random response in first 50 rounds
np.random.seed(42)
num_initial_rounds = 50
initial_data = {
    'Round': np.arange(1, num_initial_rounds + 1),
    'Your_Action': np.random.choice([0, 1], size=num_initial_rounds),
    'Opponent_Action': np.random.choice([0, 1], size=num_initial_rounds)
}
df = pd.DataFrame(initial_data)

# Restrict opponent actions to 25 cheats in first 50 rounds
opponent_cheat_count = 0
for i in range(num_initial_rounds):
    if opponent_cheat_count < 25 and df.at[i, 'Opponent_Action'] == 0:
        opponent_cheat_count += 1
    else:
        df.at[i, 'Opponent_Action'] = 1

# Prepare initial model
df['Your_Prev_Action'] = df['Your_Action'].shift(1).fillna(0)
df['Round'] = pd.Categorical(df['Round'], categories=all_rounds)
X = df[['Round', 'Your_Prev_Action']]
y = df['Opponent_Action']
X = pd.get_dummies(X, columns=['Round'], drop_first=True)
model.fit(X, y)

def make_decision(current_round, your_last_action):
    if current_round <= 50:
        # First 50 rounds: Random response with cheat limit
        if np.random.rand() < 0.5 and opponent_cheats < CHEAT_LIMIT:
            return 0
        else:
            return 1
    else:
        # Rounds 51-100: Use Linear Regression model
        current_data = pd.DataFrame({
            'Round': [current_round],
            'Your_Prev_Action': [your_last_action]
        })
        current_data['Round'] = pd.Categorical(current_data['Round'], categories=all_rounds)
        current_data = pd.get_dummies(current_data, columns=['Round'], drop_first=True)
        
        # Ensure all columns are present
        for col in X.columns:
            if col not in current_data.columns:
                current_data[col] = 0
        
        predicted_opponent_action = model.predict(current_data)[0]
        if opponent_cheats >= CHEAT_LIMIT:
            return 1
        return 1 if predicted_opponent_action > 0.7 else 0

def update_model():
    global model, X
    df_new = pd.DataFrame({
        'Round': np.arange(1, len(player_actions) + 1),
        'Your_Action': player_actions,
        'Opponent_Action': opponent_actions
    })
    df_new['Your_Prev_Action'] = df_new['Your_Action'].shift(1).fillna(0)
    df_new['Round'] = pd.Categorical(df_new['Round'], categories=all_rounds)
    X_new = df_new[['Round', 'Your_Prev_Action']]
    y_new = df_new['Opponent_Action']
    X_new = pd.get_dummies(X_new, columns=['Round'], drop_first=True)
    model.fit(X_new, y_new)
    X = X_new  # Update the global X variable

def draw_text(text, pos, color=BLACK, font=FONT):
    text_surface = font.render(text, True, color)
    text_rect = text_surface.get_rect(center=pos)
    screen.blit(text_surface, text_rect)

def draw_instructions_screen():
    screen.fill(WHITE)
    draw_text("Game Instructions", (SCREEN_WIDTH / 2, SCREEN_HEIGHT * 0.1), color=BLACK, font=SCORE_FONT)
    draw_text("Welcome to the Cooperation vs Cheating Game!", (SCREEN_WIDTH / 2, SCREEN_HEIGHT * 0.2), color=BLACK, font=FONT)
    draw_text("Controls:", (SCREEN_WIDTH / 2, SCREEN_HEIGHT * 0.3), color=BLACK, font=FONT)
    draw_text("Press 'C' to Cooperate", (SCREEN_WIDTH / 2, SCREEN_HEIGHT * 0.35), color=BLACK, font=FONT)
    draw_text("Press 'H' to Cheat", (SCREEN_WIDTH / 2, SCREEN_HEIGHT * 0.4), color=BLACK, font=FONT)
    draw_text("Concept:", (SCREEN_WIDTH / 2, SCREEN_HEIGHT * 0.5), color=BLACK, font=FONT)
    draw_text("Each round, you can choose to Cooperate or Cheat.", (SCREEN_WIDTH / 2, SCREEN_HEIGHT * 0.55), color=BLACK, font=FONT)
    draw_text("Cooperate to share resources and gain mutual rewards.", (SCREEN_WIDTH / 2, SCREEN_HEIGHT * 0.6), color=BLACK, font=FONT)
    draw_text("Cheat to gain more at the expense of your opponent.", (SCREEN_WIDTH / 2, SCREEN_HEIGHT * 0.65), color=BLACK, font=FONT)
    draw_text("First 50 rounds are a demo to understand the game.", (SCREEN_WIDTH / 2, SCREEN_HEIGHT * 0.75), color=BLACK, font=FONT)
    draw_text("Press any key to start the demo rounds.", (SCREEN_WIDTH / 2, SCREEN_HEIGHT * 0.85), color=BLACK, font=FONT)
    pygame.display.flip()

def draw_demo_screen():
    screen.fill(WHITE)
    draw_text("Demo Rounds (1-50)", (SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 - 50), color=BLACK, font=SCORE_FONT)
    draw_text("First 50 rounds are for understanding the game.", (SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2), color=BLACK, font=FONT)
    draw_text("Press any key to start the demo rounds.", (SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 + 50), color=BLACK, font=FONT)
    pygame.display.flip()

def draw_end_screen():
    screen.fill(WHITE)
    if player_score > opponent_score:
        draw_text("You Win!", (SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2), color=GREEN, font=SCORE_FONT)
    elif player_score < opponent_score:
        draw_text("Opponent Wins!", (SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2), color=RED, font=SCORE_FONT)
    else:
        draw_text("It's a Tie!", (SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2), color=BLACK, font=SCORE_FONT)
    draw_text(f"Final Score - You: {player_score} Opponent: {opponent_score}", (SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 + 50), color=BLACK, font=FONT)
    pygame.display.flip()
    pygame.time.wait(5000)

def game_loop():
    global round_num, player_score, opponent_score, player_cheats, opponent_cheats
    running = True
    instructions_displayed = False
    demo_screen_displayed = False

    while running:
        screen.fill(WHITE)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.KEYDOWN:
                if not instructions_displayed:
                    instructions_displayed = True
                    continue
                if not demo_screen_displayed:
                    demo_screen_displayed = True
                    continue

                if round_num <= 100:
                    player_action = 1 if event.key == pygame.K_c else 0
                    opponent_action = make_decision(round_num, player_actions[-1] if player_actions else 0)

                    if player_action == 0 and player_cheats >= CHEAT_LIMIT:
                        draw_text("Cheat limit reached!", (SCREEN_WIDTH / 2, SCREEN_HEIGHT * 0.5), color=RED, font=SCORE_FONT)
                        pygame.display.flip()
                        pygame.time.wait(2000)
                        continue
                    
                    player_actions.append(player_action)
                    opponent_actions.append(opponent_action)
                    results.append("Cooperate" if opponent_action == 1 else "Cheat")
                    
                    if opponent_action == 0:
                        opponent_cheats += 1
                    if player_action == 0:
                        player_cheats += 1
                    
                    if player_action == 1 and opponent_action == 1:
                        player_score += 4
                        opponent_score += 4
                    elif player_action == 1 and opponent_action == 0:
                        player_score -= 1
                        opponent_score += 3
                    elif player_action == 0 and opponent_action == 1:
                        player_score += 3
                        opponent_score -= 1
                    elif player_action == 0 and opponent_action == 0:
                        player_score += 2
                        opponent_score += 2
                    
                    round_num += 1
                    
                    if round_num == 51:
                        player_score = 0
                        opponent_score = 0
                        player_cheats = 0  # Reset player cheat count
                        opponent_cheats = 0  # Reset opponent cheat count

                    update_model()
        
        if not instructions_displayed:
            draw_instructions_screen()
        elif not demo_screen_displayed:
            draw_demo_screen()
        elif round_num > 100:
            draw_end_screen()
            running = False
        else:
            draw_text(f"Your Score: {player_score}", (SCREEN_WIDTH / 2, SCREEN_HEIGHT * 0.1), color=GREEN, font=SCORE_FONT)
            draw_text(f"Opponent Score: {opponent_score}", (SCREEN_WIDTH / 2, SCREEN_HEIGHT * 0.2), color=RED, font=SCORE_FONT)
            draw_text(f"Round: {round_num}", (SCREEN_WIDTH / 2, SCREEN_HEIGHT * 0.3), color=BLACK, font=SCORE_FONT)
            draw_text(f"Your Cheats: {player_cheats}/{CHEAT_LIMIT}", (SCREEN_WIDTH / 2, SCREEN_HEIGHT * 0.4), color=BLACK, font=SCORE_FONT)
            draw_text(f"Opponent Cheats: {opponent_cheats}/{CHEAT_LIMIT}", (SCREEN_WIDTH / 2, SCREEN_HEIGHT * 0.5), color=BLACK, font=SCORE_FONT)

        pygame.display.flip()
        pygame.time.wait(100)

    pygame.quit()

if __name__ == "__main__":
    game_loop()


pygame 2.5.2 (SDL 2.28.3, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
